# DP1 target fields summary

The notebook computes statistics and summaries of all the data in the 7 DP1 fields. It computes a number of parameters to beuse din the paper<br>

Tables produced:<br> 
* Table 2: Summary of the DP1 fields.<br>
* Table X: tracts per fields for the datapriducts section <br>


In [1]:
# Generic python packages
import os
import csv
import requests
import yaml
import numpy as np
import pandas as pd

# LSST Science Pipelines 
from lsst.daf.butler import Butler

# Set rubin publication plot style
from lsst.utils.plotting import publication_plots, get_multiband_plot_colors
import matplotlib.pyplot as plt

## Setup

### Publication style

In [2]:
# Setup publication style
publication_plots.set_rubin_plotstyle()
colors = get_multiband_plot_colors()
bands = colors.keys()  # important to get the right order for plot legends
bands_dict = publication_plots.get_band_dicts()
%matplotlib inline

Set up Rubin matplotlib plot style.
This includes dicts for colors (bandpass colors for white background),
  colors_black (bandpass colors for black background), symbols, and line_styles,
  keyed on band (ugrizy).


In [23]:
# Custom formatter: no trailing zeros if not needed
def custom_float(x):
    if isinstance(x, float):
        s = f"{x:.3f}" #.rstrip('0').rstrip('.')
        return s
    return x

SyntaxError: invalid syntax (3237341801.py, line 4)

In [24]:
# Function to round to N significant figures
def round_sf(x, sig=3):
    return np.round(x, sig - int(np.floor(np.log10(abs(x)))) - 1)

### DP1 Butler

In [5]:
instrument = 'LSSTComCam'
collections = ['LSSTComCam/DP1/defaults', 
               'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260',
               'skymaps', ]
skymap = 'lsst_cells_v1'
butler = Butler("/repo/dp1",
                instrument=instrument, 
                collections=collections, 
                skymap=skymap)
registry = butler.registry
skymap = butler.get('skyMap', skymap=skymap)

### ConsDB for metadata

In [6]:
#How to get target metadata for all the visits?
# butler or consdb?

In [7]:
# Extract filter and pointing information about the fileds from the DP1 exposures
exposures = registry.queryDimensionRecords('exposure')
exp_df = pd.DataFrame(columns=['id', 'target', 'physical_filter','ra', 'dec'])
for count, info in enumerate(exposures):
    try:
        exp_df.loc[count] = [info.id, info.target_name, info.physical_filter, 
                         info.tracking_ra, info.tracking_dec]
    except: 
        print(">>>   Unexpected error:", sys.exc_info()[0])


In [8]:
# Physical filter -> band
exp_df['band'] = exp_df['physical_filter'].str.split('_').str[0]

In [9]:
(exp_df)

,id,target,physical_filter,ra,dec,band
0,2024110800245,slew_icrs,i_06,53.327236,-28.072055,i
1,2024110800246,slew_icrs,r_03,53.327425,-28.072347,r
2,2024110800247,slew_icrs,r_03,53.141378,-28.131211,r
3,2024110800248,slew_icrs,i_06,53.141304,-28.131213,i
4,2024110800249,slew_icrs,i_06,53.188483,-28.208666,i
...,...,...,...,...,...,...
1787,2024121100607,Seagull,r_03,106.262882,-10.700520,r
1788,2024121100608,Seagull,r_03,106.272920,-10.496384,r
1789,2024121100609,Seagull,r_03,106.360054,-10.420939,r
1790,2024121100610,Seagull,r_03,106.394204,-10.512472,r


In [10]:
# slew_icrs is the ECDFS field -- combine them
exp_df.loc[exp_df['target'] == 'slew_icrs', 'target'] = 'ECDFS'

In [11]:
# List of unique fields in DP1
dp1_fields = exp_df.target.unique()
assert len(dp1_fields) == 7

In [12]:
# Number of images for each field, total and per band
n_images_all_bands = exp_df.groupby('target').size()
n_images_all_bands = n_images_all_bands.rename('Total').reset_index()

In [13]:
# Compute the number of exposures per band and set the standard band order
n_images_per_band  = exp_df.groupby(['target', 'band']).size().unstack(fill_value=0)[bands]
assert n_images_per_band.to_numpy().sum() == exposures.count()

In [14]:
n_images_summary = pd.merge(n_images_per_band, n_images_all_bands, on='target', how='left')
n_images_summary

,target,u,g,r,i,z,y,Total
0,47_Tuc,6,10,32,19,0,5,72
1,ECDFS,43,230,237,162,153,30,855
2,EDFS_comcam,20,61,87,42,42,20,272
3,Fornax_dSph,0,5,25,12,0,0,42
4,Rubin_SV_095_-25,33,82,84,23,60,10,292
5,Rubin_SV_38_7,0,44,40,55,20,0,159
6,Seagull,10,37,43,0,10,0,100


In [15]:
# Manually insert a description column
n_images_summary.insert(1, 'Field Name',[
    "47 Tucanae Globular Cluster",
    "Extended Chandra Deep Field South",
    "Rubin SV Euclid Deep Field South",
    "Fornax Dwarf Spheroidal Galaxy",
    "Rubin SV Low Galactic Latitude Field",
    "Rubin SV Low Ecliptic Latitude Field",
    "Seagull Nebula"])
n_images_summary

,target,Field Name,u,g,r,i,z,y,Total
0,47_Tuc,47 Tucanae Globular Cluster,6,10,32,19,0,5,72
1,ECDFS,Extended Chandra Deep Field South,43,230,237,162,153,30,855
2,EDFS_comcam,Rubin SV Euclid Deep Field South,20,61,87,42,42,20,272
3,Fornax_dSph,Fornax Dwarf Spheroidal Galaxy,0,5,25,12,0,0,42
4,Rubin_SV_095_-25,Rubin SV Low Galactic Latitude Field,33,82,84,23,60,10,292
5,Rubin_SV_38_7,Rubin SV Low Ecliptic Latitude Field,0,44,40,55,20,0,159
6,Seagull,Seagull Nebula,10,37,43,0,10,0,100


In [16]:
# Compute the median of all pointings per field for the 
# pointing centers and add to the summary table
target_centers = exp_df.groupby('target')[['ra', 'dec']].agg(['median'])
target_centers.columns = ['_'.join(col) for col in target_centers.columns]
target_centers
target_centers = target_centers.map(lambda x: round_sf(x, 4))
dp1_targets_summary = pd.merge(n_images_summary, target_centers, on="target")

In [17]:
dp1_targets_summary

,target,Field Name,u,g,r,i,z,y,Total,ra_median,dec_median
0,47_Tuc,47 Tucanae Globular Cluster,6,10,32,19,0,5,72,6.128,-72.090
1,ECDFS,Extended Chandra Deep Field South,43,230,237,162,153,30,855,53.160,-28.100
2,EDFS_comcam,Rubin SV Euclid Deep Field South,20,61,87,42,42,20,272,59.150,-48.730
3,Fornax_dSph,Fornax Dwarf Spheroidal Galaxy,0,5,25,12,0,0,42,40.080,-34.450
4,Rubin_SV_095_-25,Rubin SV Low Galactic Latitude Field,33,82,84,23,60,10,292,95.040,-25.000
5,Rubin_SV_38_7,Rubin SV Low Ecliptic Latitude Field,0,44,40,55,20,0,159,37.980,7.015
6,Seagull,Seagull Nebula,10,37,43,0,10,0,100,106.300,-10.510


In [18]:
# Rename and reorder 
dp1_targets_summary.rename(columns={
    'target': "Field Code",
    'ra_median': 'RA',
    'dec_median': 'DEC'}, inplace=True)

# Reorder columns by specifying the new order
dp1_targets_summary.insert(2, 'RA', dp1_targets_summary.pop('RA'))
dp1_targets_summary.insert(3, 'DEC', dp1_targets_summary.pop('DEC'))


,Field Code,Field Name,RA,DEC,u,g,r,i,z,y,Total
0,47_Tuc,47 Tucanae Globular Cluster,6.128,-72.090,6,10,32,19,0,5,72
1,ECDFS,Extended Chandra Deep Field South,53.160,-28.100,43,230,237,162,153,30,855
2,EDFS_comcam,Rubin SV Euclid Deep Field South,59.150,-48.730,20,61,87,42,42,20,272
3,Fornax_dSph,Fornax Dwarf Spheroidal Galaxy,40.080,-34.450,0,5,25,12,0,0,42
4,Rubin_SV_095_-25,Rubin SV Low Galactic Latitude Field,95.040,-25.000,33,82,84,23,60,10,292
5,Rubin_SV_38_7,Rubin SV Low Ecliptic Latitude Field,37.980,7.015,0,44,40,55,20,0,159
6,Seagull,Seagull Nebula,106.300,-10.510,10,37,43,0,10,0,100


In [19]:
# Format the date to produce a latex table
# Insert blank colum for nice spacing in table 
dp1_targets_summary.insert(4, ' ', ' ')

# Escape the underscores in the field names and codes
dp1_targets_summary.columns = dp1_targets_summary.columns.str.replace('_', r'\_', regex=False)
dp1_targets_summary = dp1_targets_summary.map(lambda x: x.replace('_', r'\_') if isinstance(x, str) else x)

dp1_targets_summary_latex = dp1_targets_summary.to_latex(index=False, 
                                                         escape=False, 
                                                         bold_rows=False,
                            formatters={col: custom_float 
                                        for col in dp1_targets_summary.columns}
                            )

# Remove unnecessary latex
data_latex = dp1_targets_summary_latex.split("midrule\n", 1)[-1]  # Keeps the part after 'midrule'
data_latex = data_latex.split("\\bottomrule", 1)[0]  # Keeps the part before 'bottomrule'

In [22]:
dp1_targets_summary

,Field Code,Field Name,RA,DEC,,u,g,r,i,z,y,Total
0,47\_Tuc,47 Tucanae Globular Cluster,6.128,-72.090,,6,10,32,19,0,5,72
1,ECDFS,Extended Chandra Deep Field South,53.160,-28.100,,43,230,237,162,153,30,855
2,EDFS\_comcam,Rubin SV Euclid Deep Field South,59.150,-48.730,,20,61,87,42,42,20,272
3,Fornax\_dSph,Fornax Dwarf Spheroidal Galaxy,40.080,-34.450,,0,5,25,12,0,0,42
4,Rubin\_SV\_095\_-25,Rubin SV Low Galactic Latitude Field,95.040,-25.000,,33,82,84,23,60,10,292
5,Rubin\_SV\_38\_7,Rubin SV Low Ecliptic Latitude Field,37.980,7.015,,0,44,40,55,20,0,159
6,Seagull,Seagull Nebula,106.300,-10.510,,10,37,43,0,10,0,100


In [20]:
data_latex

'47\\_Tuc & 47 Tucanae Globular Cluster & 6.128000 & -72.090000 &   & 6 & 10 & 32 & 19 & 0 & 5 & 72 \\\\\nECDFS & Extended Chandra Deep Field South & 53.160000 & -28.100000 &   & 43 & 230 & 237 & 162 & 153 & 30 & 855 \\\\\nEDFS\\_comcam & Rubin SV Euclid Deep Field South & 59.150000 & -48.730000 &   & 20 & 61 & 87 & 42 & 42 & 20 & 272 \\\\\nFornax\\_dSph & Fornax Dwarf Spheroidal Galaxy & 40.080000 & -34.450000 &   & 0 & 5 & 25 & 12 & 0 & 0 & 42 \\\\\nRubin\\_SV\\_095\\_-25 & Rubin SV Low Galactic Latitude Field & 95.040000 & -25.000000 &   & 33 & 82 & 84 & 23 & 60 & 10 & 292 \\\\\nRubin\\_SV\\_38\\_7 & Rubin SV Low Ecliptic Latitude Field & 37.980000 & 7.015000 &   & 0 & 44 & 40 & 55 & 20 & 0 & 159 \\\\\nSeagull & Seagull Nebula & 106.300000 & -10.510000 &   & 10 & 37 & 43 & 0 & 10 & 0 & 100 \\\\\n'

In [21]:
# Export to latex with deulxetable formatting 
with open("../tables/dp1_fields_summary.tex", "w") as f:
    f.write(r"""%%%%% This table is auto generated from data, DO NOT EDIT
\begin{deluxetable}{llcccp{0.5cm}p{0.6cm}p{0.3cm}p{0.3cm}p{0.3cm}p{0.3cm}r}
\caption{DP1 fields and pointing centers with the number of images in each band per field.  
ICRS coordinates are in units of decimal degrees. 
\label{tab:dp1_fields} }
\tablehead{
  \colhead{\textbf{Field Code}} & \colhead{\textbf{Field Name}} & \colhead{\textbf{RA}} & \colhead{\textbf{DEC}} 
  & & \multicolumn{6}{c}{\textbf{Band}} & \colhead{\textbf{Total}}\\
  \cline{3-4} \cline{6-11} 
  & & \colhead{deg}  & \colhead{deg}  & & u & g & r & i & z & y & 
}
\startdata
""")
    f.write(data_latex)
    f.write(r"""\enddata
\end{deluxetable}
""")
f.close()